# What parameters are not important in our model?
In this Notebook we will apply Global Sensitivity Analysis (GSA) to identify the parameters in a model, if any, which have a negligible influence on the model results. In this way, we can reduce the number of parameters and hence the time and money that we need to invest to improve our model. In this example we will apply GSA to a hydrological model.

# But what is a hydrological model?
Imagine that we want to simulate the natural water flows draining into a river. Knowing the amount of rainfall that has fallen in the river drainage area, i.e. catchment, we can estimate how much of the rainwater flows over (surface flow) and under (subsurface flow) the land surface and finally reach the river. 

<left><img src="util/catchment.png" width="500px">

For this purpose, we can use a hydrological model. A hydrological model is a mathematical model (set of equations) describing the hydrological processes that occur in a catchment as a function of various parameters. These model parameters describe the hydrological characteristics of the catchment, such as the climate and soil characteristics, ultimately enabling the estimation of the river flow at selected river sections.

## Hydrological processes in a river section
This animated image represents the main hydrological processes that occur in one of the river sections. 

<left><img src="util/Rain_runoff_diagram7.gif" width="700px">
    
## Diagram of the hydrological model

<left><img src="util/HyMOD_diagram_simple4.gif" width="1000px">

## Model parameters
To represent the hydrological processes the model use mathematical equations that are as a function of several parameters.

<left><img src="util/Rain_runoff_param.png" width="500px">
    
Usually the value of the model parameters are unknown and ideally, to obtain a good representation of the hydrological processes occuring in the catchment, we should measure in-situ their values. However, this data is generally not available and it is difficult and expensive to measure. 

Imagine that have a limited budget to make in-situ measurements and we can measure two of the paramater values. Therefore, we need to carefully select which ones we would like to measure. 
    
## Calibration of the model parameters

To tailor a hydrological model to a particular catchment, model calibration is required. Model calibration is the process of adjusting the model parameters to obtain a representation of the catchment under study that satisfies pre-agreed criteria. Normally, calibration of a hydrological model aims to improve the fit of the simulated river flow to observed river flow and involves running the model many times under different combinations of the parameter values, until a combination is found that minimise the differences between simulated and observed flows. 

Let's try to calibrate the model manually, that is, changing the parameter values one at the time and looking at the effects induced in the model predictions by means of an interactive plot of the simulated hydrograph. The objective is to obtain a good fit of the simulated hydrograph to the observed one.

In [1]:
from ipywidgets import widgets
from util.hydrological_model import hydrological_model

Soil_sto,Evap_rate,Inf_rate, Time_surf,Time_under,slider_box_layout,\
           fig_sto,fig_flo,fig_hyd,hbox_layout,vbox_layout,param_obs = hydrological_model()

widgets.VBox([widgets.VBox([Soil_sto,Evap_rate,Inf_rate, Time_surf,Time_under],layout=slider_box_layout),
              widgets.HBox([fig_sto,fig_flo,fig_hyd],layout=hbox_layout)],layout = vbox_layout)

# What is Global Sensitivity Analysis? and why shall we use it?

**Global Sensitivity Analysis** is a set of mathematical techniques which investigate how uncertainty in the output of a numerical model can be attributed to variations of its input factors.

The benefits of applying GSA are:

1. **Better understanding of the model**
    
2. **Test whether the model "behaves well"**
    
3. **Identify the important inputs on which to focus computation time (e.g. for calibration), acquisition of new data, etc.** 
    
4. **Understand the main impacts of input uncertainty on the modelling outcome and thus model-informed decisions**

## How Global Sensitivity Analysis works?

GSA investigates how the uncertainty of the selected model input parameters influences the variability of the model output/performance.

A '**model parameter**' is any element that can be changed before running the model.

An '**output**' is any variable that is obtained after the model execution.

Before executing the model, we will sample the parameters from their ranges of variability and then repeatedly run the model so that for each simulation all the inputs vary simultaneously. After GSA is performed we obtain a set of sensitivity indices for each output. The sensitivity indices measure the relative influence of each input factor on the output (*Refs. 4-5*).

## Plot sensitivity indices and identify non-influential parameters
The dummy parameter is a numerical artifice, with no influence on the model output, which is used to estimate the threshold for non-influential inputs. 

Uninfluential input factors should have zero-valued sensitivity indices, but since sensitivity indices are computed by numerical approximations rather than analytical solutions, an uninfluential factor may still be associated with a non-zero (although small) index value. 

Therefore if the index of an input factor is below the value of the sensitivity index of the dummy parameter, then the input factor is deemed uninfluential *(Ref. 6)*.

In [2]:
from util.GSA_hydrological_model import screening, mapping

x1,x2,x3,x4,x5,fig1 = screening(param_obs)

widgets.VBox([widgets.HBox([widgets.VBox([x1,x2,x3,x4,x5],layout=dict(border='solid',width = '475px',height = '180px')),fig1])])

## Let's try to calibrate the model parameters again...
Now try to calibrate the hydrological model, i.e. minimize the difference between the simulated flow and the observed flow, but now neglecting the parameters that according to GSA are not influential. Can you reduce the difference between the simulated and observed flows to a value **below 85** by only **changing the value of 2 parameters**?

In [3]:
Soil_sto,Evap_rate,Inf_rate, Time_surf,Time_under,slider_box_layout,\
           fig_sto,fig_flo,fig_hyd,hbox_layout,vbox_layout,param_obs = hydrological_model()

widgets.VBox([widgets.VBox([Soil_sto,Evap_rate,Inf_rate, Time_surf,Time_under],layout=slider_box_layout),
              widgets.HBox([fig_sto,fig_flo,fig_hyd],layout=hbox_layout)],layout = vbox_layout)

### References

1. [SAFE Website](https://www.safetoolbox.info/)
2. [Introductory paper to SAFE - Pianosi et al. (2015)](https://www.sciencedirect.com/science/article/pii/S1364815215001188)
3. [PAWN method - Pianosi and Wagener (2018)](https://doi.org/10.1016/j.envsoft.2018.07.019)
4. [A review of available methods and workflows for Sensitivity Analysis - Pianosi et al. (2016)](https://www.sciencedirect.com/science/article/pii/S1364815216300287)
5. [What has Global Sensitivity Analysis ever done for us? A systematic review to support scientific advancement and to inform policy-making in earth system modelling - Wagener and Pianosi (2019)](https://www.sciencedirect.com/science/article/pii/S0012825218300990)
6. [Global Sensitivity Analysis . The primer - Saltelli et al. (2008)](http://www.andreasaltelli.eu/file/repository/A_Saltelli_Marco_Ratto_Terry_Andres_Francesca_Campolongo_Jessica_Cariboni_Debora_Gatelli_Michaela_Saisana_Stefano_Tarantola_Global_Sensitivity_Analysis_The_Primer_Wiley_Interscience_2008_.pdf) 
7. [Dummy parameter - Zadeh et al. (2017)](https://www.sciencedirect.com/science/article/pii/S1364815217301159)